## To perform the analysis, we need two data resouces:
### 1. Blood pressure, using tables chartevents, d_items and services
### 2. Potassium level, using tables labevents, d_labitems and services

In [19]:
%matplotlib inline
import matplotlib
matplotlib.style.use('ggplot')
matplotlib.rcParams['figure.figsize'] = (10.0, 8.0)

import pandas as pd
import numpy as np
pd.options.display.max_columns = 10
pd.options.display.max_rows = 10

import warnings
warnings.filterwarnings('ignore')
from chatto_transform.sessions.mimic import mimic_common

## Get blood pressure from chartevents
### SQL query:

#### 1. Get itemid from d_items for blood pressure measurements:

SELECT *
	FROM MIMICIII.D_ITEMS
	WHERE label ILIKE 'FEM ART'
	    OR label ILIKE 'FEMORAL SHEATH'
	    OR label ILIKE 'ABP Alarm'
	    OR label ILIKE 'ABP'
	    OR label ILIKE 'Arterial BP Mean'
	    OR label ILIKE 'Fem sheath MAP'
	    OR label ILIKE 'Femoral ABP'
	    OR label ILIKE 'map'
	    OR label ILIKE 'Arterial Blood Pressure mean'
	    OR label ILIKE 'Non Invasive Blood Pressure mean'
	    OR label ILIKE 'Non-Invasive Blood Pressure Alarm - Low'
	    OR label ILIKE 'art mean'
	    OR label ILIKE 'arterial mean #2'

#### 2. Get chartevents rows for blood presure, quick query (save to csv file)

In [ ]:
from sqlalchemy.sql import text
from chatto_transform.sessions.mimic import mimic_login
from chatto_transform.datastores.sqlalchemy_datastore import fast_postgresql_to_csv

In [ ]:
# SQL query, limited to "arterial blood pressure mean" from carevue (itemid=52)
# To get metavision, use itemid=220052
# To get everything related to blood pressure, change to 
# "WHERE itemid in (220052,438,52,1321,5845,2543,2642,2647,2294,2731,6590,6605,2233752,220181)"
# The command below save query results into local disk

query = text("""
SELECT * FROM
    (SELECT * FROM mimiciii.chartevents 
    WHERE itemid=52) as C

    JOIN 
        (SELECT subject_id, hadm_id, transfertime
        FROM mimiciii.services
            WHERE curr_service = 'CMED') AS cmed
    ON C.subject_id = cmed.subject_id
""").compile()

engine = mimic_login.get_engine()
query.bind = engine

directory = mimic_login.get_local_storage_dir()
fast_postgresql_to_csv(query, directory+'/blood_pressure.csv')


#### 3. Get potassium level of patients admitted to CMED 

In [ ]:
# SQL query 
# pull data from the labevents, itemid = 50971 (potassium, blood, chemistry)

SELECT cmed.subject_id, cmed.hadm_id AS cmed_hadm_id, cmed.transfertime AS cmed_transfertime,
   lab.charttime AS k_charttime, lab.value AS k_value, lab.valuenum AS k_valuenum, lab.uom AS k_uom, lab.hadm_id AS k_hadm_id, lab.itemid AS k_itemid
FROM
(SELECT subject_id, hadm_id, transfertime
   FROM 
   mimiciii.services
   WHERE curr_service = 'CMED') AS cmed
LEFT JOIN mimiciii.labevents AS lab
   ON lab.subject_id = cmed.subject_id
   AND lab.itemid = 50971

In [ ]:
mimic_common.sql()

In [ ]:
# save data
df = mimic_common.loaded_sql[0]
df.to_csv('ktable.csv')